In [21]:
# List of cities to analyze
cities = ['madrid, spain']

# Initialize storage
phi_means = []  # Store mean Phi values per city
phi_stds = []  # Store standard deviation per city
phi_values = []  # Store full list of Phi values per city
all_faces = {}  # Store extracted faces per city
all_face_areas = {}  # Store face areas per city

for city in cities:
    print('--------- City: ', city)

    # Download road network
    if city == "manSeg, NY":
        G = ox.graph_from_point((40.77892619481051, -73.9510420715686), 300, network_type="drive")
    else:
        G = ox.graph_from_place(city, network_type="drive")

    # Convert to undirected and extract edges
    G_undirected = nx.Graph(G)
    gdf_edges = ox.graph_to_gdfs(G, nodes=False, edges=True)
    gdf_edges = gdf_edges.to_crs(epsg=3857)  # Convert to metric projection

    # Create a list of LineStrings representing the street edges
    edge_lines = [LineString([(G.nodes[u]['x'], G.nodes[u]['y']), (G.nodes[v]['x'], G.nodes[v]['y'])]) for u, v in G_undirected.edges()]

    # Extract the enclosed faces using polygonization
    faces = list(polygonize(edge_lines))
    faces = gpd.GeoSeries(faces, crs='EPSG:4326').to_crs(epsg=3857)  # Convert polygons to metric projection
    all_faces[city] = faces  # Store faces per city

    city_phi_values = []  # Store Phi values for the current city
    face_areas = []  # Store block areas for the current city

    print(f"Number of faces in {city}: {len(faces)}")

    for polygon in tqdm(faces, desc=f"Processing faces for {city}"):
        A = polygon.area  # Block area
        face_areas.append(A / 10000)  # Convert m² to hectares

        # Compute the largest internal distance (D)
        coords = list(polygon.exterior.coords)
        max_dist = 0
        for i, p1 in enumerate(coords):
            for p2 in coords[i+1:]:
                dist = np.linalg.norm(np.array(p1) - np.array(p2))
                max_dist = max(max_dist, dist)
        D = max_dist  # Maximum distance

        if D > 0:
            phi = 4 * A / (np.pi * D**2)
            city_phi_values.append(phi)  # Store all values for later analysis

    # Store the computed values per city
    phi_means.append(np.mean(city_phi_values) if city_phi_values else 0)
    phi_stds.append(np.std(city_phi_values, ddof=1) if len(city_phi_values) > 1 else 0)  # Compute standard deviation
    phi_values.append(city_phi_values if city_phi_values else [])  # Store full list of values
    all_face_areas[city] = face_areas

# Print summary statistics
for city, mean, std in zip(cities, phi_means, phi_stds):
    print(f"{city}: Mean Phi = {mean:.3f}, Phi Std = {std:.3f}, Phi Median = {np.median(phi_values[cities.index(city)]):.3f}")

# Calculate and print mode for each city
for city, phi_list in zip(cities, phi_values):
    rounded_values = np.round(phi_list, decimals=3)
    mode_result = scipy.stats.mode(rounded_values)
    mode_value = float(mode_result.mode)
    print(f"{city}: Phi Mode = {mode_value:.3f}")

# Print IQR and MAD for phi values
for city, phi_list in zip(cities, phi_values):
    phi_array = np.array(phi_list)
    q1 = np.percentile(phi_array, 25)
    q3 = np.percentile(phi_array, 75)
    iqr = q3 - q1
    mad = np.mean(np.abs(phi_array - np.mean(phi_array)))
    print(f"\n{city} additional statistics:")
    print(f"IQR = {iqr:.3f}")
    print(f"MAD = {mad:.3f}")

# Print face area summary statistics
print("\n=== FACE AREA STATISTICS (square meters) ===")
for city, areas in all_face_areas.items():
    areas_m2 = np.array(areas) * 10000
    mean_area = np.mean(areas_m2)
    median_area = np.median(areas_m2)
    std_area = np.std(areas_m2)
    mad_area = np.mean(np.abs(areas_m2 - mean_area))
    q1_area = np.percentile(areas_m2, 25)
    q3_area = np.percentile(areas_m2, 75)
    iqr_area = q3_area - q1_area
    rounded_areas = np.round(areas_m2, -2)
    mode_result = scipy.stats.mode(rounded_areas)
    mode_area = float(mode_result.mode)
    
    print(f"\n{city} face area statistics:")
    print(f"Number of faces: {len(areas_m2)}")
    print(f"Mode = {mode_area:.1f} m²")
    print(f"Mean = {mean_area:.1f} m²")
    print(f"Median = {median_area:.1f} m²")
    print(f"Std = {std_area:.1f} m²")
    print(f"MAD = {mad_area:.1f} m²")
    print(f"IQR = {iqr_area:.1f} m²")

--------- City:  madrid, spain


NameError: name 'ox' is not defined

In [ ]:
# Print all phi values
print("=== ALL PHI VALUES ===")
for city, phi_list in zip(cities, phi_values):
    print(f"\nPhi values for {city}:")
    print(f"Number of values: {len(phi_list)}")
    sorted_values = sorted(phi_list)
    print("Values:", [f"{x:.3f}" for x in sorted_values])

# Print all face areas
print("\n=== ALL FACE AREAS (square meters) ===")
for city, areas in all_face_areas.items():
    areas_m2 = np.array(areas) * 10000
    print(f"\n{city} all face areas:")
    print(f"Number of faces: {len(areas_m2)}")
    print("Areas (m²):")
    print('\n'.join([f"{x:.1f}" for x in sorted(areas_m2)]))